# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd



# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
salaries.head(3)


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
salaries.isnull().sum()


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
salaries["Salary or Hourly"].value_counts()


Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
salaries["Department"].value_counts()



Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
import scipy.stats as st
import numpy as np 

In [11]:
##H0: mu hour rate =30/h
##H1: mu hour rate != 30/h 
alpha = 0.05
hourly_wages = salaries['Hourly Rate'].dropna()
c_sample = hourly_wages.sample(2000)
c_sample.head()

14550    19.50
16850    13.94
9294     40.20
31614    18.52
21968    35.60
Name: Hourly Rate, dtype: float64

In [13]:
mean = c_sample.mean()
std = c_sample.std(ddof=1)

stat = (mean - 30)/ (std/ np.sqrt(2000))
print(stat)

9.213189919496925


In [14]:
p_value = st.t.sf(abs(stat), 2000 -1) *2
if p_value > 0.05:
    print ("I cannot reject the null hypothesis")
else:
    print("We can reject the null hypothesis") 

We can reject the null hypothesis


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [15]:
c2_sample = salaries[salaries['Department'] =="POLICE"].sample(30)
c2_sample.head()



,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
15571,"KNIGHTEN, FREDERICK",DETENTION AIDE,POLICE,F,Salary,NaN,64392.0,NaN
5726,"CORDIN, CHARLES",POLICE OFFICER,POLICE,F,Salary,NaN,84054.0,NaN
22098,"OSHANA, SARGON",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
13712,"JACKSON IV, WESLEY J",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
2756,"BRACKIN, CONNOR",POLICE OFFICER,POLICE,F,Salary,NaN,76266.0,NaN


In [19]:
c2_sample_f = c2_sample['Annual Salary']
mean = c2_sample_f.mean()
std = c2_sample_f.std(ddof=1)
mu = 86000

stat = (mean - 86000)/ (std/ np.sqrt(30))
p_value = st.t.sf(abs(stat), 30-1) *2
if p_value > 0.05:
    print ("I cannot reject the null hypothesis")
else:
    print("We can reject the null hypothesis") 

I cannot reject the null hypothesis


Using the `crosstab` function, find the department that has the most hourly workers. 

In [24]:
hourly_data = salaries[salaries["Salary or Hourly"] == "Hourly"]

In [31]:

cross_tab = pd.crosstab(salaries["Department"], salaries["Department"]).dropna()

hourly_counts = cross_tab[hourly_data]

# Find the department with the maximum hourly workers
most_hourly_department = hourly_counts.idxmax()

print("Department with the most hourly workers:", most_hourly_department)



Department with the most hourly workers: Department
ADMIN HEARNG            NaN
ANIMAL CONTRL           NaN
AVIATION                NaN
BOARD OF ELECTION       NaN
BOARD OF ETHICS         NaN
BUDGET & MGMT           NaN
BUILDINGS               NaN
BUSINESS AFFAIRS        NaN
CITY CLERK              NaN
CITY COUNCIL            NaN
COMMUNITY DEVELOPMENT   NaN
COPA                    NaN
CULTURAL AFFAIRS        NaN
DISABILITIES            NaN
DoIT                    NaN
FAMILY & SUPPORT        NaN
FINANCE                 NaN
FIRE                    NaN
GENERAL SERVICES        NaN
HEALTH                  NaN
HUMAN RELATIONS         NaN
HUMAN RESOURCES         NaN
INSPECTOR GEN           NaN
LAW                     NaN
LICENSE APPL COMM       NaN
MAYOR'S OFFICE          NaN
OEMC                    NaN
POLICE                  NaN
POLICE BOARD            NaN
PROCUREMENT             NaN
PUBLIC LIBRARY          NaN
STREETS & SAN           NaN
TRANSPORTN              NaN
TREASURER               

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [17]:
most_hourly_workers_department = salaries['Department'].value_counts().idxmax()
print("Department with the most hourly workers:", most_hourly_workers_department)

Department with the most hourly workers: POLICE


In [ ]:
import scipy.stats as stats


target_department = 'POLICE'
target_hourly_wages = salaries.loc[salaries['department'] == target_department, 'Hourly Rate']

null_hypothesis_mean = 35

# Perform the one-sample t-test
t_statistic, p_value = stats.ttest_1samp(target_hourly_wages, null_hypothesis_mean)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Check if the p-value is less than 0.05 for a one-sided test
alpha = 0.05
if p_value < alpha:
    print(f"At the 95% confidence level, we reject the null hypothesis.")
else:
    print(f"At the 95% confidence level, we fail to reject the null hypothesis.")




# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [21]:
# Your code here:

import numpy as np
from scipy.stats import t
from scipy.stats import sem

hourly_rate = salaries['Hourly Rate'].dropna()

confidence_interval = st.t.interval(0.95, len(hourly_rate)-1, loc=hourly_rate.mean(), scale=st.sem(hourly_rate))
confidence_interval


(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [23]:
# Your code here:
hourly_rate_police = salaries.loc[salaries['Department'] == 'POLICE', 'Hourly Rate'].dropna()

confidence_interval = st.t.interval(0.95, len(hourly_rate_police) - 1, loc=hourly_rate_police.mean(), scale=st.sem(hourly_rate_police))
confidence_interval


(9.31381234362183, 9.45418765637817)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

